In [1]:
## CV TESTING GROUNDS ##

import numpy as np
import math
import cv2
import time

In [90]:
# Control Variables

VIDEO = "SampleVideo_1280x720_2mb.mp4"
IMAGE_RESOLUTION_RESIZE = (600,520)

bright_threshold = 30
focus_threshold = 5000
  
# define color boundaries to filtering highlight (BGR)
color_filter_boundaries = [([120, 120, 120], [250, 250, 250])]

TRACKER = "CSRT"
BRIGHT_DETECTION = 0
FOCUS_DETECTION = 0
COLOR_FILTER = 1
WATCH_FILTER = 1
CANNY = 1
WATCH_CANNY = 1
TRACK_CANNY = 1
EROSION = 1
DILATION = 0
BODY_CENTROID = 0
ELLIPSE_DETECTION = 0
SPLIT_FOCUS = 0
SHOW_GRID = 0
GRID_SIZE = 30
SHOW_FPS = 1

# Camera Properties

'''
IMAGE_RESOLUTION = (1280, 1024)
ZOOM_LEVEL = 0
RESOLUTION_MAX_ZOOM = 0.16 # microns/pixel
RESOLUTION_MIN_ZOOM = 1.90 # microns/pixel
FIELD_OF_VIEW_MAX_ZOOM =  (0.38, 0.24) # mm
FIELD_OF_VIEW_MIN_ZOOM =  (2.4, 1.9) # mm
'''

'\nIMAGE_RESOLUTION = (1280, 1024)\nZOOM_LEVEL = 0\nRESOLUTION_MAX_ZOOM = 0.16 # microns/pixel\nRESOLUTION_MIN_ZOOM = 1.90 # microns/pixel\nFIELD_OF_VIEW_MAX_ZOOM =  (0.38, 0.24) # mm\nFIELD_OF_VIEW_MIN_ZOOM =  (2.4, 1.9) # mm\n'

In [31]:
def bright_detection(frame, gray_frame, bright_Threshold):

    blur = cv2.blur(gray_frame , (5, 5))

    a, b, c, d = cv2.mean(blur)

    if a > bright_Threshold:
        cv2.putText(frame, "BRIGHT", (470, 15),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 80, 80), 1)
        
    else:
        cv2.putText(frame, "DARK", (470, 15),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 80, 80), 1)

In [32]:
def focus_detection(frame, gray_frame, focus_threshold):
    
    # Focus Analyser Using Scharr Function
    focus = cv2.Scharr(gray_frame, cv2.CV_64F, 1, 0).var() # Apply Scharr;
        
    # Print if is Focused;
    if (focus > focus_threshold):
        cv2.putText(frame, "FOCUSED", (470, 40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        return 1

    # Print if not Focused;
    else:
        cv2.putText(frame, "NO FOCUS", (470, 40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        return 0

In [33]:
def split_focus(gray_frame):
    
    h = int(gray_frame.shape[0]/2)
    w = int(gray_frame.shape[1]/2)
        
    (y1, x1, y2, x2, y3, x3, y4, x4) = (0, 0, 0, w, h, 0, h, w)

    upleft_quad = gray_frame[y1:y1+h, x1:x1+w]
    upright_quad = gray_frame[y2:y2+h, x2:x2+w]
    downleft_quad = gray_frame[y3:y3+h, x3:x3+w]
    downright_quad = gray_frame[y4:y4+h, x4:x4+w]

    focus1 = cv2.Scharr(upleft_quad, cv2.CV_64F, 1, 0).var() # Apply Scharr;
    focus2 = cv2.Scharr(upright_quad, cv2.CV_64F, 1, 0).var() # Apply Scharr;
    focus3 = cv2.Scharr(downleft_quad, cv2.CV_64F, 1, 0).var() # Apply Scharr;
    focus4 = cv2.Scharr(downright_quad, cv2.CV_64F, 1, 0).var() # Apply Scharr;

    cv2.putText(upleft_quad, "Focus: {}".format(focus1), (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1) 
    cv2.putText(upright_quad, "Focus: {}".format(focus2), (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1) 
    cv2.putText(downleft_quad, "Focus: {}".format(focus3), (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1) 
    cv2.putText(downright_quad, "Focus: {}".format(focus4), (20, 20),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1) 

    cv2.imshow("Up Left Quad", upleft_quad)
    cv2.imshow("Up Right Quad", upright_quad)
    cv2.imshow("Down Left Quad", downleft_quad)
    cv2.imshow("Down Right Quad", downright_quad)

In [34]:
def color_filtering(frame, boundaries):

    # loop over the boundaries
    for (lower, upper) in boundaries:

        # create NumPy arrays from the boundaries
        lower = np.array(lower, dtype = "uint8") # Lower color limit
        upper = np.array(upper, dtype = "uint8") # Upper color limit

        mask = cv2.inRange(frame, lower, upper) # mask with objects in range of lower to upper
        output_filter = cv2.bitwise_and(frame, frame, mask = mask) # output = roiselect & mask
        output_filter[np.where((output_filter == [0,0,0]).all(axis = 2))] = [255,255,255] # the background is black, turn it to white
        output_filter[np.where((output_filter != [255,255,255]).all(axis = 2))] = [0,0,0] # turn everything that is not white to black
        output_filter = cv2.cvtColor(output_filter,cv2.COLOR_BGR2GRAY)
            
        output_filter = cv2.bitwise_not(output_filter)
        
    return output_filter
    

In [35]:
def canny_custom(gray_frame, erosion, dilation):
    
    if (erosion > 0):
        kernel = np.ones((5,5),np.uint8)
        gray_frame = cv2.erode(gray_frame,kernel,iterations = erosion)
           
    blurred = cv2.GaussianBlur(gray_frame, (3, 3), 1)
    frame_canny = cv2.Canny(blurred, 10, 10)
        
    if (dilation > 0):
        kernel = np.ones((5,5),np.uint8)
        frame_canny = cv2.dilate(frame_canny,kernel,iterations = dilation)
        frame_canny = cv2.bitwise_not(frame_canny)
        
    return frame_canny

In [36]:
def object_centroid(frame, gray_frame, canny_on, frame_canny):
    
    contours, hierarchy = cv2.findContours(gray_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if (canny_on !=0):
        contours, hierarchy = cv2.findContours(frame_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for c in contours:
        # calculate moments for each contour
        M = cv2.moments(c)
    
        # calculate x,y coordinate of centroid
        if M["m00"] != 0:
            centroidX = int(M["m10"] / M["m00"])
            centroidY = int(M["m01"] / M["m00"])
            area = cv2.contourArea(c)
            cv2.circle(frame, (centroidX, centroidY), 3, (255, 0, 0), -1)

In [37]:
def find_ellipse(frame, gray_frame, canny_on, frame_canny):
    
        contours, hierarchy = cv2.findContours(gray_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        if (CANNY !=0):
            contours, hierarchy = cv2.findContours(frame_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        
        if len(contours) != 0:
                for c in contours:
                    if (cv2.contourArea(c) > 100):
                        ellipse = cv2.fitEllipse(c)
                        (x, y), (MA, ma), angle = ellipse
                        EllipseArea = (PI / 4 * MA * ma)
                        if (350 > x > 250):
                            if ((MA*1.7 > ma > MA*0.3) or (ma*1.7 > MA > ma*0.3)) & (fig_area > EllipseArea > fig_area*0.15):

                                M = cv2.moments(c)

                                # calculate x,y coordinate of centroid
                                if M["m00"] != 0:
                                    centroidX = int(M["m10"] / M["m00"])
                                    centroidY = int(M["m01"] / M["m00"])
                                    area = cv2.contourArea(c)
                                    
                                if ((x - 6) < centroidX < (x + 6)) & ((y - 6) < centroidY < (y + 6)):
                                    
                                    cv2.ellipse(frame,ellipse,(255,200,0),2)
                                    cv2.circle(frame, (int(x), int(y)), 3, (255, 200, 0), -1)
                                    cv2.putText(frame, "{:.2f}".format(angle), (int(x)+3, int(y)+3),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 200, 0), 1)
                                    cv2.circle(frame, (centroidX, centroidY), 3, (255, 0, 0), -1)

In [92]:
# Starting Setup:

# Create MultiTracker object
Tracker = cv2.MultiTracker_create()

# ROI State Variable
TRACKING_ON = 0

PI = np.pi

# Select Desired Video
video = cv2.VideoCapture(VIDEO)

# Video stream loop:

while True:
    ret, frame = video.read()
    
    start = time.time()
    
    if not ret: # Break the loop if no video is load.
        break
    
    frame = cv2.resize(frame,(IMAGE_RESOLUTION_RESIZE))
    fig_area = (frame.shape[0]*frame.shape[1])
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #**************************************************************************************************************************
    #**********************************************/Bright Detection/**********************************************************
    if (BRIGHT_DETECTION == 1):
        
        bright_detection(frame, gray_frame, bright_threshold)
      
    #**************************************************************************************************************************
    #**********************************************/Focus Detection/***********************************************************
    
    # Focus Variance threshold, must be calibrated using a focused sample;
    
    FOCUS = 1
    
    if (FOCUS_DETECTION == 1):
    
        FOCUS = focus_detection(frame, gray_frame, focus_threshold)
         
    #**************************************************************************************************************************
    #**************************************/Split in Quadrants Focus Detection/************************************************ 
    if (SPLIT_FOCUS == 1):
        
        split_focus(gray_frame)

    #**************************************************************************************************************************
    #********************************************/Color Filter Application/****************************************************
    if (COLOR_FILTER == 1):
        
        gray_frame = color_filtering(frame, color_filter_boundaries)
        output_filter_show = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
        
    #**************************************************************************************************************************
    #**********************************************/Canny Application/*********************************************************
    if (CANNY == 1):
        
        frame_canny = canny_custom(gray_frame, EROSION, DILATION)
        frame_canny_show = cv2.cvtColor(frame_canny, cv2.COLOR_GRAY2BGR)

    #**************************************************************************************************************************
    #**********************************************/Ellipse Detection/*********************************************************
    if (ELLIPSE_DETECTION == 1) & (FOCUS == 1):

        if (CANNY == 0):
            find_ellipse(frame, gray_frame, canny_on, frame_canny)
        
        if (CANNY == 1):
            find_ellipse(frame, gray_frame, canny_on, frame_canny)

    #**************************************************************************************************************************
    #**********************************************/Body Centroid/*************************************************************                   
    if (BODY_CENTROID == 1):
        
        if (CANNY == 0):
            object_centroid(frame, gray_frame, CANNY, gray_frame)
        
        if (CANNY == 1):
            object_centroid(frame, gray_frame, CANNY, frame_canny)
                      
    #**************************************************************************************************************************
    #**********************************************/Selected ROI Tracker/******************************************************
    
    # Press P to Select the Region of Interest and Track it using CSRT Tracker.
    # Press K after a tracking start to stop it.
    # Press Q to break the video stream and close windows.
    
    key = cv2.waitKey(1) or 0xff
    
    if (FOCUS != 1):
        cv2.putText(frame, "Not ready to Track, adjust focus.", (100, 510),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
    if (FOCUS == 1) & (TRACKING_ON == 0):
        cv2.putText(frame, "Ready to Track, press P to select the Region of Interest.", (100, 510),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    #************************************* PRESS Q TO CLOSE STREAM ************************************************************
    #**************************************************************************************************************************
    if key == ord('q'):
        break
    #**************************************************************************************************************************
    
    #************************************* PRESS P TO SELECT TRACKING OBJECT **************************************************
    #**************************************************************************************************************************
    if key == ord('p'):
        
         if (FOCUS == 1) & (TRACKING_ON == 0):
            cv2.putText(frame, "Ready to Track, press P to select the Region of Interest.", (100, 510),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 80, 80), 1)

            if (TRACK_CANNY == 0):
                TRACKING_ON = 1
                r = cv2.selectROI(frame)

                # Add Tracker
                Tracker.add(cv2.TrackerCSRT_create(), frame, r)
               
            if (TRACK_CANNY == 1):
                TRACKING_ON = 1
                r = cv2.selectROI(frame_canny)

                # Add Tracker
                Tracker.add(cv2.TrackerCSRT_create(), frame_canny, r)
            
    #**************************************************************************************************************************
    #**************************************************************************************************************************
    
    if (TRACKING_ON == 1):
        
        # get updated location of objects in subsequent frames
        success, boxes = Tracker.update(frame)

        # draw tracked objects
        for i, newbox in enumerate(boxes):
            p1 = (int(newbox[0]), int(newbox[1]))
            p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
            cv2.rectangle(frame, p1, p2, (0, 0, 255), 2, 1)
            cX = int(p1[0] + (p2[0]-p1[0])/2)
            cY = int(p1[1] + (p2[1]-p1[1])/2)
            cv2.circle(frame, (cX, cY), 3, (0, 0, 255), -1)
            cv2.putText(frame, "CSRT", (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            cv2.putText(frame, "Object Centroid", (30,60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            cv2.putText(frame, "X {}".format(cX), (30, 90),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)   
            cv2.putText(frame, "Y {}".format(cY), (30, 120),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            
            CenterX = frame.shape[1]/2 
            CenterY = frame.shape[0]/2
            
            cv2.putText(frame, "Image Center", (470,60), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            cv2.putText(frame, "X {}".format(CenterX), (470, 90),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)   
            cv2.putText(frame, "Y {}".format(CenterY), (470, 120),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            
            #******************************************************************************************************************
            #**************************************/ Centralization Checking /*************************************************
            PosBand = 6

            DX = (CenterX - cX)
            DY = (CenterY - cY)

            cv2.putText(frame, "DELTA X {}".format(DX), (470, 270),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(frame, "DELTA Y {}".format(DY), (470, 300),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(frame, "Tolerance: {}".format(PosBand), (470, 330),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            # Check if is Centralized

            if (cX > (CenterX - PosBand)) & (cX < (CenterX + PosBand)):
                cv2.putText(frame, "Centralized X", (470, 200),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            if (cY > (CenterY - PosBand)) & (cY < (CenterY + PosBand)):
                cv2.putText(frame, "Centralized y", (470, 230),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                
    #**************************************************************************************************************************
    #**************************************************************************************************************************

    #************************************* PRESS K TO BREAK CURRENTLY TRACKING ************************************************
    #**************************************************************************************************************************
    if key == ord('k'):
        TRACKING_ON = 0
        Tracker = cv2.MultiTracker_create() # clear tracking object
    #**************************************************************************************************************************
    
    #************************************* SHOW GRID **************************************************************************
    if (SHOW_GRID == 1):

        height, width, channels = frame.shape
        for x in range(0, width -1, GRID_SIZE):
            cv2.line(frame, (x, 0), (x, height), (255, 255, 255), 1, 1)

        for y in range(0, width -1, GRID_SIZE):
            cv2.line(frame, (0, y), (width, y), (255, 255, 255), 1, 1)
    
    #************************************* SHOW VIDEO STREAM ******************************************************************
       
    if (SHOW_FPS == 1):
        end = time.time()

        frame_time = (end - start) + 0.0001
        fps = np.floor(1/frame_time)

        cv2.putText(frame, "FPS: {}".format(fps), (0, 430),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 50, 50), 2)
    
    if (WATCH_FILTER != 1) & (WATCH_CANNY != 1):
        cv2.imshow("ROI selector", frame)
    
    if (WATCH_FILTER != 1) & (WATCH_CANNY == 1):
        canny_h_concat = np.concatenate((frame, frame_canny_show), axis=1)
        cv2.imshow("ROI selector", canny_h_concat)
        
    if (WATCH_FILTER == 1) & (WATCH_CANNY != 1):
        filter_h_concat = np.concatenate((frame, output_filter_show), axis=1)
        cv2.imshow("ROI selector", filter_h_concat)
        
    if (WATCH_FILTER == 1) & (WATCH_CANNY == 1):
        watch_1 = np.concatenate((frame, output_filter_show), axis=0)
        watch_2 = np.concatenate((frame_canny_show,  frame), axis=0)
        all_watch = np.concatenate((watch_1, watch_2), axis=1)
        cv2.imshow("ROI selector", all_watch)
    
    #**************************************************************************************************************************    
        
video.release()
cv2.destroyAllWindows()